In [29]:
# Import
import numpy as np
import pandas as pd
import napari
import numpy as np
from scipy.interpolate import RBFInterpolator

import tifffile 


In [30]:
# Load lut with em centroids to transform (csv file)
lut = pd.read_csv(r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220426_RM0008_130hpf_fP1_f3\bigwarp_alignment\lut_centroids.csv')
lut.head()

,id,em_centroids_zyx_px
0,1,"(292.07773851590105, 2.097173144876325, 34.254..."
1,2,"(216.02410939186757, 2.485066570708888, 40.270..."
2,3,"(290.19642857142856, 2.044642857142857, 36.955..."
3,4,"(299.05970149253733, 1.7014925373134329, 42.16..."
4,5,"(300.7578947368421, 2.3157894736842106, 36.726..."


In [31]:
em_centroids = np.array([eval(t) for t in lut['em_centroids_zyx_px']])
print(em_centroids.shape)

(11288, 3)


In [32]:
# Load landmarks to create transform
lmks_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220426_RM0008_130hpf_fP1_f3\bigwarp_alignment\20240710_mov_EM_fix_LM_landmarks.csv'

In [33]:
lmks = pd.read_csv(lmks_path, header= None)
# Add the headers as shown in the image
headers = ['Name', 'Active', 'mvg-x', 'mvg-y', 'mvg-z', 'fix-x', 'fix-y', 'fix-z']

# Assign the headers to the DataFrame
lmks.columns = headers
lmks.head()

,Name,Active,mvg-x,mvg-y,mvg-z,fix-x,fix-y,fix-z
0,Pt-0,False,15.401238,23.064857,67.894922,8.473757,50.299048,48.974068
1,Pt-1,False,16.572585,36.465325,98.828049,35.424851,66.547966,79.553103
2,Pt-2,False,51.741957,17.723752,93.737745,67.471004,42.754333,74.472040
3,Pt-5,False,43.160962,38.318928,39.673012,41.138765,51.146687,6.435591
4,Pt-6,False,38.967385,36.177572,37.005390,35.281416,47.460107,9.680567


In [34]:
em_landmarks = lmks[lmks['Active']==True][['mvg-z','mvg-y','mvg-x']].to_numpy()/[.16,0.4,0.16]
lm_landmarks = lmks[lmks['Active'] == True][['fix-z', 'fix-y', 'fix-x']].to_numpy()/[.4,0.4,0.4]
print(em_landmarks.shape)

(351, 3)


In [35]:
# Setting up the RBFInterpolator for TPS
source_landmarks = em_landmarks
target_landmarks = lm_landmarks
# The 'thin_plate' kernel is used for TPS
tps_kernel = 'thin_plate_spline'

# Instantiate the RBFInterpolator for each dimension
rbf_interpolator = RBFInterpolator(source_landmarks, target_landmarks, kernel=tps_kernel)

In [36]:

# Apply the transformation using the RBFInterpolator
transformed_landmarks = rbf_interpolator(em_landmarks)
transformed_centroids = rbf_interpolator(em_centroids)
# Save lut 

In [37]:
# Visualize
vv = napari.Viewer()
#landmarks
vv.add_points(em_landmarks, face_color='m')
vv.add_points(lm_landmarks, face_color='y')
vv.add_points(transformed_landmarks , face_color='c')
#centroids
vv.add_points(em_centroids, face_color='r')
vv.add_points(transformed_centroids, face_color='b')

D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Points layer 'transformed_centroids' at 0x2bb359cb7f0>

In [38]:
lm_stack = tifffile.imread(r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220426_RM0008_130hpf_fP1_f3\bigwarp_alignment\lm_stack.tif') 
vv.add_image(lm_stack, name='lm_stack')



<Image layer 'lm_stack' at 0x2bab298e980>

In [39]:

from scipy.ndimage import map_coordinates
em_mask = tifffile.imread(r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220426_RM0008_130hpf_fP1_f3\bigwarp_alignment\fine_aligned_downsampled_4_em_stack_cropped_woResin_rough_rotated_to_LM_cp_masks_cp-1_ft00.tif')

vv.add_labels(em_mask, name='em_mask')

<Labels layer 'em_mask' at 0x2bb44080d30>

In [40]:
# save transformed_em_centroids as lm_stack_processed_centroids
# Convert the numpy array to a list of tuples if it's not already
centroid_list = [tuple(centroid) for centroid in transformed_centroids]

# Create a DataFrame with unique IDs and centroids
lut['lm_stack_processed_centroids_zyx_px'] = centroid_list

lut.head()

,id,em_centroids_zyx_px,lm_stack_processed_centroids_zyx_px
0,1,"(292.07773851590105, 2.097173144876325, 34.254...","(75.89187233237317, 56.922545552828524, -62.94..."
1,2,"(216.02410939186757, 2.485066570708888, 40.270...","(40.53236700830166, 47.28642468712496, -71.049..."
2,3,"(290.19642857142856, 2.044642857142857, 36.955...","(74.74128570254834, 56.385826600791916, -61.85..."
3,4,"(299.05970149253733, 1.7014925373134329, 42.16...","(78.41970488730294, 56.72814055383424, -57.770..."
4,5,"(300.7578947368421, 2.3157894736842106, 36.726...","(79.64862928491493, 58.07835549408628, -60.075..."


In [41]:

lut.to_csv(r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220426_RM0008_130hpf_fP1_f3\bigwarp_alignment\lut_centroids.csv', index=False)
